In [ ]:
from datasets import load_from_disk
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

import json
from typing import Iterable, Dict, List
from tqdm.notebook import tqdm

from radon.metrics import mi_visit

import signal

In [9]:
# Define a custom exception for timeouts
class TimeoutException(Exception):
    pass

# Define the handler function for the signal
def timeout_handler(signum, frame):
    raise TimeoutException("Function call timed out")

# Set the signal handler for SIGALRM
signal.signal(signal.SIGALRM, timeout_handler)

<function __main__.timeout_handler(signum, frame)>

In [10]:
def test_one_example(example):
    temp_namespace = {}

    try:
        exec(example["code"], temp_namespace)
    except Exception as e:
        print(f"Error executing code for example id {example['task_id']}: {e}")
        return [False, ["Code execution error"]]

    # print(f"Running tests for example id: {example['task_id']}")

    failed_tests = []

    all_tests_passed = True
    for i, test in enumerate(example['test_list']):
        try:
            signal.alarm(5)  # Set the alarm for 5 seconds
            exec(test, temp_namespace)
            signal.alarm(0)  # Disable the alarm
            # print(f"Test {i+1} passed.")
        except AssertionError as e:
            all_tests_passed = False
            print(f"Test {i+1} failed.")
            print(f"  -> AssertionError: {e}")
            failed_tests.append(i+1)
        except TimeoutException as e:
            all_tests_passed = False
            print(f"Test {i+1} failed.")
            print(f"  -> Timeout: {e}")
            failed_tests.append(i+1)
        except Exception as e:
            all_tests_passed = False
            print(f"Test {i+1} failed.")
            print(f"  -> Exception: {e}")
            failed_tests.append(i+1)
        finally:
            signal.alarm(0)  # Disable the alarm always
    
    del temp_namespace

    return [all_tests_passed, failed_tests] # Empty list if all tests passed

In [11]:
# Load test dataset
test_dataset = load_from_disk("./mbpp_test_with_signatures")

In [12]:
dfs = []

In [13]:
for run_number in range(1, 6):
    print(f"--- Processing Run {run_number} ---")
    
    # Define file paths based on run number
    input_json = f"gemma_1b_dpo_250_further_250_model_output_run_{run_number}.json"
    output_parquet = f"gemma_1b_dpo_250_further_250_model_output_analysis_run_{run_number}.parquet"

    # Load results json
    with open(input_json, "r") as f:
        result_baseline = json.load(f)

    # columns: task_id, output_1, output_2, output_3, output_4, output_5, output_6, output_7, output_8, output_9, output_10
    # Create a DataFrame from scratch
    df = pd.DataFrame(columns=["task_id", "prompt", "test_list", "output_1", "output_2", "output_3", "output_4", "output_5",
                               "output_6", "output_7", "output_8", "output_9", "output_10"])

    # Load into DataFrame
    for item in result_baseline:
        row = {"task_id": item["task_id"]}
        for i, code in enumerate(item["generated_sequences"]):
            row[f"output_{i+1}"] = code
        
        # Find task in test dataset
        test_entry = next((entry for entry in test_dataset if entry["task_id"] == item["task_id"]), None)
        if test_entry:
            row["test_list"] = test_entry["test_list"]
            row["prompt"] = test_entry["text"]
        
        df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

    # Test against test dataset to check functional correctness
    for i, row in df.iterrows():
        task_id = row["task_id"]
        # Find the corresponding entry in the test dataset
        for j in range(1, 11):
            output_col = f"output_{j}"
            print(f"Testing task_id {task_id}, output {j}")
            if pd.notna(row[output_col]):
                example = {
                    "task_id": task_id,
                    "code": row[output_col].replace("```python", "").replace("```", "").strip(),
                    "test_list": row["test_list"]
                }

                result = test_one_example(example)
                
                df.at[i, f"output_{j}_pass"] = str(result)
            else:
                df.at[i, f"output_{j}_pass"] = str([False, [k for k in range(1, len(row["test_list"])+1)]])  # No code means all tests fail

    for i, row in df.iterrows():
        # Count number of outputs that passed all tests
        pass_count = 0
        for j in range(1, 11):
            output_col = f"output_{j}_pass"
            if pd.notna(row[output_col]):
                result = eval(row[output_col])  # Convert string back to list
                if result[0]:  # If all tests passed
                    pass_count += 1
        df.at[i, "num_outputs_all_tests_passed"] = pass_count
        if pass_count >= 1:
            df.at[i, "pass@10"] = True
        else:
            df.at[i, "pass@10"] = False

    # Calculate pass@10 percentage
    total_tasks = len(df)
    tasks_with_pass = df["pass@10"].sum()
    pass_at_10_percentage = (tasks_with_pass / total_tasks) * 100
    print(f"Pass@10 for run {run_number}: {pass_at_10_percentage:.2f}% ({tasks_with_pass}/{total_tasks})")

    # Compute maintainability index for generated inputs that passed all tests
    for i, row in df.iterrows():
        for j in range(1, 11):
            output_col = f"output_{j}"
            output_pass_col = f"output_{j}_pass"
            result = eval(row[output_pass_col])
            if result[0]:  # If all tests passed
                code = row[output_col].replace("```python", "").replace("```", "").strip()
                mi_score = mi_visit(code, True)
                df.at[i, f"output_{j}_mi"] = mi_score
            else:
                df.at[i, f"output_{j}_mi"] = None

    # Add an average mi_score column
    mi_cols = [f"output_{j}_mi" for j in range(1, 11)]
    df["avg_mi_score"] = df[mi_cols].mean(axis=1)

    # Print average mi_score of avg_mi_score column
    if not df["avg_mi_score"].dropna().empty:
        overall_avg_mi = df["avg_mi_score"].dropna().mean()
        print(f"Overall Average Maintainability Index for run {run_number} across {df['avg_mi_score'].count()} passed outputs: {overall_avg_mi:.2f}")

    # Save DataFrame to parquet
    df.to_parquet(output_parquet, index=False)
    print(f"Saved results for run {run_number} to {output_parquet}")

    dfs.append(df)

--- Processing Run 1 ---
Testing task_id 11, output 1
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 11, output 2
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 11, output 3
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 11, output 4
Test 1 failed.
  -> AssertionError: 
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 11, output 5
Test 1 failed.
  -> AssertionError: 
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 11, output 6
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 11, output 7
Test 1 failed.
  -> AssertionError: 
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 11, output 8
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 11, output 9
Test 1 fail

<string>:20: RuntimeWarning: invalid value encountered in sqrt
<string>:21: RuntimeWarning: invalid value encountered in sqrt
<string>:29: RuntimeWarning: invalid value encountered in arccos
<string>:28: RuntimeWarning: invalid value encountered in arccos
<string>:23: RuntimeWarning: invalid value encountered in sqrt
<string>:24: RuntimeWarning: invalid value encountered in sqrt


Test 1 failed.
  -> Timeout: Function call timed out
Test 2 failed.
  -> Timeout: Function call timed out
Test 3 failed.
  -> Timeout: Function call timed out
Testing task_id 260, output 3
Test 1 failed.
  -> AssertionError: 
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> Timeout: Function call timed out
Testing task_id 260, output 4
Test 1 failed.
  -> AssertionError: 
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 260, output 5
Test 1 failed.
  -> Exception: name 'newman_shanks_williams_prime' is not defined
Test 2 failed.
  -> Exception: name 'newman_shanks_williams_prime' is not defined
Test 3 failed.
  -> Exception: name 'newman_shanks_williams_prime' is not defined
Testing task_id 260, output 6
Error executing code for example id 260: unterminated triple-quoted string literal (detected at line 7) (<string>, line 2)
Testing task_id 260, output 7
Test 1 failed.
  -> Timeout: Function call timed out
Test 2 failed.
  -> Timeout: Fu

<string>:23: RuntimeWarning: invalid value encountered in sqrt
<string>:22: RuntimeWarning: invalid value encountered in sqrt
<string>:23: RuntimeWarning: invalid value encountered in sqrt
<string>:24: RuntimeWarning: invalid value encountered in sqrt
<string>:20: RuntimeWarning: invalid value encountered in sqrt
<string>:21: RuntimeWarning: invalid value encountered in sqrt
<string>:18: RuntimeWarning: invalid value encountered in sqrt
<string>:19: RuntimeWarning: invalid value encountered in sqrt
<string>:20: RuntimeWarning: invalid value encountered in sqrt


Test 1 failed.
  -> AssertionError: 
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 300, output 7
Test 1 failed.
  -> AssertionError: 
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 300, output 8
Test 1 failed.
  -> AssertionError: 
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 300, output 9
Test 1 failed.
  -> AssertionError: 
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 300, output 10
Test 1 failed.
  -> AssertionError: 
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 301, output 1
Test 1 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 301, output 2
Testing task_id 301, output 3
Testing task_id 301, output 4
Testing task_id 301, output 5
Testing task_id 301, output 6
Error executing code for example id 301: unmatched ')' (<string>, line

...

Test 1 failed.
  -> Timeout: Function call timed out
Test 2 failed.
  -> Timeout: Function call timed out
Test 3 failed.
  -> Timeout: Function call timed out
Testing task_id 260, output 4
Test 1 failed.
  -> AssertionError: 
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 260, output 5
Test 1 failed.
  -> Exception: name 'newman_shanks_williams_prime' is not defined
Test 2 failed.
  -> Exception: name 'newman_shanks_williams_prime' is not defined
Test 3 failed.
  -> Exception: name 'newman_shanks_williams_prime' is not defined
Testing task_id 260, output 6
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 260, output 7
Test 1 failed.
  -> AssertionError: 
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 260, output 8
Test 1 failed.
  -> AssertionError: 
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 260, output 9
Test 2 f

<string>:15: RuntimeWarning: invalid value encountered in sqrt
<string>:16: RuntimeWarning: invalid value encountered in sqrt
<string>:17: RuntimeWarning: invalid value encountered in sqrt
<string>:18: RuntimeWarning: invalid value encountered in sqrt
<string>:19: RuntimeWarning: invalid value encountered in sqrt
<string>:20: RuntimeWarning: invalid value encountered in sqrt
<string>:19: RuntimeWarning: invalid value encountered in sqrt
<string>:20: RuntimeWarning: invalid value encountered in sqrt
<string>:22: RuntimeWarning: invalid value encountered in sqrt
<string>:23: RuntimeWarning: invalid value encountered in sqrt


Test 1 failed.
  -> Timeout: Function call timed out
Test 2 failed.
  -> Timeout: Function call timed out
Test 3 failed.
  -> Timeout: Function call timed out
Testing task_id 261, output 1
Test 1 failed.
  -> AssertionError: 
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 261, output 2
Test 1 failed.
  -> AssertionError: 
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 261, output 3
Test 1 failed.
  -> AssertionError: 
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 261, output 4
Testing task_id 261, output 5
Test 1 failed.
  -> AssertionError: 
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 261, output 6
Test 1 failed.
  -> AssertionError: 
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 261, output 7
Test 1 failed.
  -> AssertionError: 
Test 2 failed.
  -> AssertionError: 
Te

<string>:22: RuntimeWarning: invalid value encountered in sqrt
<string>:18: RuntimeWarning: invalid value encountered in sqrt
<string>:19: RuntimeWarning: invalid value encountered in sqrt
<string>:20: RuntimeWarning: invalid value encountered in sqrt
<string>:18: RuntimeWarning: invalid value encountered in sqrt
<string>:19: RuntimeWarning: invalid value encountered in sqrt
<string>:20: RuntimeWarning: invalid value encountered in sqrt
<string>:18: RuntimeWarning: invalid value encountered in sqrt
<string>:19: RuntimeWarning: invalid value encountered in sqrt
<string>:18: RuntimeWarning: invalid value encountered in sqrt
<string>:19: RuntimeWarning: invalid value encountered in sqrt


Test 1 failed.
  -> Timeout: Function call timed out
Test 2 failed.
  -> Timeout: Function call timed out
Test 3 failed.
  -> Timeout: Function call timed out
Testing task_id 260, output 4
Test 1 failed.
  -> AssertionError: 
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 260, output 5
Test 1 failed.
  -> AssertionError: 
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 260, output 6
Test 1 failed.
  -> Exception: name 'newman_shanks_williams_prime' is not defined
Test 2 failed.
  -> Exception: name 'newman_shanks_williams_prime' is not defined
Test 3 failed.
  -> Exception: name 'newman_shanks_williams_prime' is not defined
Testing task_id 260, output 7
Test 1 failed.
  -> AssertionError: 
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Testing task_id 260, output 8
Test 1 failed.
  -> Timeout: Function call timed out
Test 2 failed.
  -> Timeout: Function call timed out
Test 3

In [14]:
for run_number, df in enumerate(dfs, start=1):
    total_tasks = len(df)
    tasks_with_pass = df["pass@10"].sum()
    pass_at_10_percentage = (tasks_with_pass / total_tasks) * 100
    print(f"Run {run_number} - Pass@10: {pass_at_10_percentage:.2f}% ({tasks_with_pass}/{total_tasks})")

    mi_cols = [f"output_{j}_mi" for j in range(1, 11)]
    valid_mi_scores = df[mi_cols].stack().dropna()

    if not df["avg_mi_score"].dropna().empty:
        overall_avg_mi = df["avg_mi_score"].dropna().mean()
        print(f"Run {run_number} - Average Maintainability Index across {df['avg_mi_score'].count()} passed outputs: {overall_avg_mi:.2f}")
    else:
        print(f"Run {run_number} - No passed outputs with maintainability index available.")

Run 1 - Pass@10: 44.80% (224/500)
Run 1 - Average Maintainability Index across 224 passed outputs: 88.88
Run 2 - Pass@10: 44.80% (224/500)
Run 2 - Average Maintainability Index across 224 passed outputs: 88.90
Run 3 - Pass@10: 45.00% (225/500)
Run 3 - Average Maintainability Index across 225 passed outputs: 88.89
Run 4 - Pass@10: 45.00% (225/500)
Run 4 - Average Maintainability Index across 225 passed outputs: 88.67
Run 5 - Pass@10: 44.60% (223/500)
Run 5 - Average Maintainability Index across 223 passed outputs: 88.63
